Notities lesson 4

* CONVOLUTIONAL LAYERS zijn layers die een bepaalde filter elementwise met de input layer vermeningvuldigen en opgeteld. conv2d heeft als input het aantal filters, gevolgd door de afmetingen van die filters (conv2D(2,(3,3),..)). Bij een tweede conv layer heb je 1 filter dat een 3,3,2 tensor is. Dit is dus equivalent aan twee 3,3 matrices die elk met een van de vorige filter uitkomsten wordt vermenigvuldigd en opgeteld. 
* MAXPOOL LAYERS, decreases resolution of the previous layer. When to maxpool? depends. Why do we do maxpooling? 1 helps with translational invariance; if the image is slightly shifted, it helps. 2 it reduces the size of the input so that a 3,3 filter covers a relatively larger area. geoffrey hinton capsule is needed for further development of maxpooling.
* DENSE layers zijn layers die Hadamard (elementwise) vermenigvuldigd worden met alle waardes van de vorig matrix (en hebben dus dezelfde vorm). de uitkomst van een dense layer is de som van alle filterxactivation opgeteld. Jeremy heeft het ook wel over activation. Ik denk omdat de vermenigvuldigde waardes bij elkaar worden opgeteld om zo tot een 'probability' te komen.
* INSTEAD OF RANDOM WEIGHTS, USE DERIVATIVES FOR FILTERS&DENSELAYERS AND CALCULATE HOW IT IMPACTS THE LOSS.
* Difference softmax and sigmoid a^xi/SUM(a^xi), sigmoid is binary version of softmax.
* 15:00 Keras author said maybe conv networks are the best solution.
* OPTIMIZERS, check excel met alle optimizers
* 16:00 SGD, SGD gokt waardes, neemt de afgeleide, vermenigvuldigd de afgeleide met een learning rate en voegt die stap toe aan de oude waardes.
* 25:00 Relu heeft geen analytical derivative maar je kan approximaten.
* SGD verbeteren door:
    * learning rate groter; exploded
    * MOMENTUM: Neem het gemiddelde tussen de afgelopen steps en tel die in een bepaalde verhouding op bij de nieuwe gradient stap. momentum is de verhouding tussen de vorige en d huidige stap. momentum is de verhouding gradient vorige stap / gradient huidige stap. Grotere momentum betekent meer geschiedenis.
    * New problem: we need different learning rates for different parameters, fix by dynamic learning rates
    * DYNAMIC LEARNING RATES: ADAGRAD past per epoch de learning rate aan van alle variabelen. Elke epoch wordt learning rate gedeeld door de sum of squared errors (gemiddelde afgeleide waar groter, veel zwaarder weegt) per variabele. -> lr can be larger! 
    * DYNAMIC LEARNING RATES: RMSprop, mooiere versie van adagrad, coursera course van Geoffrey Hinton. RMSprop doet hetzelfde als momentum maar dan squared van de gradients per mini-batch. Dus gradient^2*0.1+gradient(vorige)*0.9. Deel lr door dit getal.
        * Implementatie: LEARNING RATE ANNEALING: run een paar keer, deel lr door 10, run een paar keer, etc
    * DYNAMIC LEARNING RATES: ADAM is een combinatie van RMSprop en momentum. Even higher learning rate!
    * DYNAMIC LEARNING RATES: EVE is een toevoeging aan adam maar dan met learning rate annealing. Het houdt de rms van de afgelopen epochs bij en deelt de lr daardoor.
    * DYNAMIC LEARNING RATES: JEREMY automatische learning rate aanpassing door als de gradient ratio verdubbeld, deel de lr door 4

# Approach to problem
* START TRAINING ON SAMPLE (check statefarm notebook, snappen dat je met een kleine dataset al iets kan leren, snappen dat het grootste gedeelte van het leren in de dense layers gebeurt, precomputing input layer)
    * LINEAR MODEL
        * Beginnen met het aller makkelijkste model; single dense layer met batchnorm en flatten ervoor. Dit geeft je referentie voor als je moeilijkere modellen gaat maken.
        * Voorbeeld met statefarm; simpel model maar wel 1,5 miljoen parameters dus niet underfitting dus het overshoot. SMALL LEARNING RATE FOR FIRST EPOCH omdat je anders op de loss function gaat leren dus als je altijd cat zegt zit je ook in de buurt. Dus eerst kleine learning rate. 1e-5 1 epoch, 1e-3 2epoch, 1e-4 
        * Hoe groot moet je sample zijn? Probeer wat uit, doe 10x evaluate en kijk hoe de val_acc varieerd. Als het weinig varieert zit je goed.
    * CONVOLUTIONAL MODEL
        * 2 conv layers met elk maxpooling en batchnorm. In dit geval was er overfitting dus data augmentation:
        * data augmentation; van elke variabele de beste waarde vinden. Het is niet duidelijk hoe je dit zou moeten doen, alles is intuitie dus probeer per augmentation variabele de goede waarde te vinden (probeer 3-4) op een groot genoege val set zodat het vrij stabiel is een combineer ze.
        * in dit geval bleef val_acc laag maar acc bleef omhoog gaan dus meer proberen
        * still overfitting; add regularization; dropout is favorite. maar voor regularization heb je de hele dataset nodig omdat add more data gecorroleerd is met regularization.
        * Nice, now with VGG: hier hoef je niet te de conv layers te trainen, die kan je pre-computen, opslaan en daarmee verder trainen. Je kan hier geen data augmentation doen. Je kan wel 5x dataaugmented input opslaan met een aangepaste train set die al gecompute is door de conv layers.
        * tried retraining some of the conv layers maar dat werkte niet. Alle interessante dingen gebeuren in de dense layers.
        * Combination van pseudo labeling [^link1] and knowledge destillation [^link2]. Pseudo labelen is je test data labelen met een prediction en vervolgens gebruiken als nieuwe test data, dit lijkt me alleen relevant voor Kaggle. Knowledge destillation is de kennis uit meerdere netwerken samenvatten.
        * You can and should do pseudo labeleling on pseudo labeling. You should take max size of pseudo labels 1/4 - 1/3 of labeled data. Keras heeft daar geen optie voor.
    * colaborative filtering, way of doing recommender systems, used for NLP
        * who is going to like what and how much? movies, products, articles on newsfeed
        * 2 classes: 1 based on genre the metadata approach, 1 based on other people, collaborative filtering (this one is way better)
        * MovieLens is een voorbeeld dataset
        * Voorbeeld van MovieLens. Users en movies krijgen elk 5 variabelen die je met dot product met elkaar vermenigvuldigd. Daar krijg je een rating uit. BIAS: verder is er nog een getal (bias) wat je bij alles optelt omdat sommige users enthousiaster zijn en sommige films enthousiaster worden beoordeeld. These variables are called LATENT FACTORS, we think they are relevant and influencing the outcome but we don't know what they are.
        * verschil SGD en GD is dat SGD per minibatch berekend en GD over de hele dataset
        * Implementatie, gebruik EMBEDDING LAYER in Keras voor snelheid, gebruik FUNCTIONAL MODEL. KAN OOK MET NEURAL NETS; CHECK LES.
        
        
    






* 1:00:00 How do you choose learning rate? You first do 1/2 epochs small learning rate, then bigger learning rate and then every time a bit smaller as soon as it starts to flatten out
* 1:08:00 how else do you approach overfitting? 

[^link1]: http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf
[^link2]: https://arxiv.org/pdf/1503.02531.pdf